## Install the package dependencies before running this notebook

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import pickle
from glob import glob
import numpy as np
import re
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as utils
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable


"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""

## Create a Torch.Dataset class for the training dataset

In [ ]:
from glob import glob
import pickle
import numpy as np

ROOT_PATH = "./"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]


device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device = 'cpu'


def standardize(inputs, outputs):
    m = np.mean(inputs, axis = (0,1), keepdims = True)
    s = np.std(np.sqrt(inputs[:, :, 0]**2 + inputs[:, :, 0]**2))
    standard_inputs = (inputs - m)/s
    standard_targets = (outputs - m)/s
    return standard_inputs, standard_targets, m, s

def standardize_test(inputs):
    m = np.mean(inputs, axis = (0,1), keepdims = True)
    s = np.std(np.sqrt(inputs[:, :, 0]**2 + inputs[:, :, 0]**2))
    standard_inputs = (inputs - m)/s
    return standard_inputs



def reverse_std(m, s, preds):
    return preds * s + m



def get_mean_std(city="palo-alto", split="train"):
    f_in = ROOT_PATH + split + "/" + city + "_inputs"
    inputs = pickle.load(open(f_in, "rb"))
    inputs = np.asarray(inputs)
    
    outputs = None
    
    if split=="train":
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)
    
    inputs, outputs, m, s = standardize(inputs, outputs)
    
    return m, s



def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    f_in = ROOT_PATH + split + "/" + city + "_inputs"
    inputs = pickle.load(open(f_in, "rb"))
    inputs = np.asarray(inputs)
    inputs = np.float32(inputs)
    
    outputs = None
    
    if split=="train":
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)
        
        #standardize
        inputs, outputs, m, s = standardize(inputs, outputs)
        outputs = np.float32(outputs)
        return inputs, outputs

    else:
        return standardize_test(inputs), None
        



class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform

        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=False)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        
        if self.outputs is None:
            data = self.inputs[idx]
        else: 
            data = (self.inputs[idx], self.outputs[idx])
            
        if self.transform:
            data = self.transform(data)

        return data

# intialize a dataset
city = 'palo-alto' 
split = 'train'
train_dataset  = ArgoverseDataset(city = city, split = split)

## Create a DataLoader class for training

In [ ]:
batch_size = 10 # batch size 
train_loader = DataLoader(train_dataset,batch_size=batch_size, drop_last = True)

## Sample a batch of data and visualize 

In [ ]:
import matplotlib.pyplot as plt
import random


def show_sample_batch(sample_batch):
    """visualize the trajectory for a batch of samples"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i,:,0], inp[i,:,1])
        axs[i].scatter(out[i,:,0], out[i,:,1])

        
for i_batch, sample_batch in enumerate(train_loader):
    inp, out = sample_batch
    """
    TODO:
      implement your Deep learning model
      implement training routine
    """
    
    show_sample_batch(sample_batch)
    break

In [ ]:
import seaborn as sns
import pandas as pd
X, Y = [], []
for x, y in train_loader:
    [[X.append(j.tolist()) for j in i] for i in x]
    [[Y.append(j.tolist()) for j in i] for i in y]

x1, x2 = [i[0] for i in X], [i[1] for i in X]
y1, y2 = [i[0] for i in Y], [i[1] for i in Y]


In [ ]:
def graph(x1, x2):
    heatmap_x, xedges, yedges = np.histogram2d(x1, x2, bins=[100, 100])
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    plt.imshow(heatmap_x.T, extent=extent, origin='lower')
    plt.show()
graph(x1,x2)

In [ ]:
graph(y1, y2)

## Build Model

In [ ]:
# Could try bidirectional hidden 
# Could try different attention mechanisms
# softmax layers and pooling layers
# normalize


In [ ]:
class encoder(nn.Module):
    
    def __init__(self, input_size, hidden_size, n_layers, dropout_p):
        super(encoder, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        #layers
        self.lstm = nn.LSTM(self.input_size, self.hidden_size,
                            self.n_layers, dropout = dropout_p, batch_first = True)
        #self.fc_hidden = nn.Linear(self.hidden_size * 2, self.hidden_size)
        #self.fc_cell = nn.Linear(hidden_size * 2, hidden_size)

    def forward(self, inp):
        encoder_output, hidden = self.lstm(inp)
        #hidden = self.fc_hidden(torch.cat((hidden[0:1], hidden[1:2]), dim=2))
        #cell = self.fc_cell(torch.cat((cell[0:1], cell[1:2]), dim=2))
        return encoder_output, hidden


In [ ]:
class decoder(nn.Module):
    
    def __init__(self, hidden_size, output_size, n_layers, dropout_p = 0.1):
        super(decoder, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.n_layers = n_layers
        
        # attention 
        self.attn = nn.Linear(hidden_size * n_layers + output_size, 50)
        self.attn_combine = nn.Linear(hidden_size + output_size, hidden_size)
        
        #layers
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size,
                            self.n_layers, dropout = dropout_p, batch_first = True)
        
        self.energy = nn.Linear(self.hidden_size * n_layers + output_size , 1)
        self.out =  nn.Linear(self.hidden_size, self.output_size)
        
        self.dropout = nn.Dropout(self.dropout_p)
        self.softmax = nn.Softmax(dim = 1)
        self.relu = nn.ReLU()
        
    def forward(self, input_, encoder_states, hidden):
        
        h = hidden[0]
        h = h.transpose(0,1).reshape(h.shape[1], -1)
        
        energy = self.attn(torch.cat([input_, h], 1))
        attention = self.softmax(energy)
                
        # usesing mps makes the encoder_state take the shape of [50, batchsize, hidden_size]
        if device == 'cpu':
            string = "bl,blh->bh"
        else: string = "bl,lbh->bh"
        context_vector = torch.einsum(string, attention, encoder_states)
        
        rnn_input = torch.cat((input_, context_vector), dim=1)
        rnn_input = self.attn_combine(rnn_input).unsqueeze(1)
        rnn_input = self.relu(rnn_input)

        output, decoder_hidden = self.lstm(rnn_input, hidden)
        output = self.out(output.float())
        
        return output.squeeze(1), decoder_hidden


In [ ]:
class model(nn.Module):
    
    def __init__(self, encoder, decoder):
        super(model, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target):
        
        batch_size = source.shape[0]
        target_len = 60

        outputs = torch.zeros(batch_size, 60, 2).to(device)
        
        encoder_states, encoder_hidden = self.encoder(source)
        decoder_hidden = encoder_hidden

        x = source[:, -1]
        for t in range(target_len):
            output, decoder_hidden = self.decoder(x, encoder_states, decoder_hidden)
            outputs[:,t] = output
        return outputs
    

## Training Routine

In [ ]:
def L2_2D(x, y):
    #x1,x2 = zip(*inp)
    #y1,y2 = zip(*out)
    #x1, x2, y1, y2 = torch.tensor(x1), torch.tensor(x2), torch.tensor(y1), torch.tensor(y2)
    #return sum(np.sqrt(np.square(x1-y1) + np.square(x2-y2)))
    return ((x - y) ** 2).sum()
    

def train(encoder, decoder, model, epochs, lr):
    
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr = lr)
    lossF = L2_2D
    
    loss_history = [np.inf]
    prev_model = None
    for epoch in range(epochs):
        
        epoch_loss = 0
        for X, y in train_loader:
            X = X.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            outputs = model(X.float(), y)
            loss = lossF(torch.reshape(outputs, [60*batch_size, 2]), torch.reshape(y, [60*batch_size, 2]))
            epoch_loss += loss
            loss.backward()
            optimizer.step()
        
        print(city + ' ' + f"Epoch: {epoch+1} Loss:{epoch_loss/len(train_loader)}")
        
        #early stopping
        if epoch_loss > (loss_history[-1] * 1.25):
            return prev_model, outputs
        
        loss_history.append(epoch_loss)
        prev_model = model
        
        
    return model, loss_history

def pred(model, test_data):
    
    t = torch.tensor([i for i in test_data])
    model.eval()
    t = t.to(device)
    fake_target = torch.zeros(len(test_data), 60, 2).to(device)
    outputs = model(t.float(), fake_target)
    return outputs

## Create Outputs

In [ ]:
#train by cities:



#universal parameters
lr = 0.002
epochs = 30
input_size = 2
hidden_size = 128
n_layers = 1
dropout_p = 0.1
output_size = 2

#logs
models = {}
loss_histories = {}
preidctions = {}


for city in cities:
    
    split = 'train'
    
    #loaders
    train_data = ArgoverseDataset(city = city, split = split)
    batch_size = 10
    train_loader = DataLoader(train_dataset,batch_size=batch_size, drop_last = True)
    
    #setup new model for each city
    encoder_net = encoder(input_size, hidden_size, n_layers, dropout_p)
    decoder_net = decoder(hidden_size, output_size, n_layers, dropout_p)
    lewis = model(encoder_net, decoder_net)
    encoder_net.to(device)
    decoder_net.to(device)
    lewis.to(device)
    
    #training
    lewis, loss_history = train(encoder_net, decoder_net, lewis, epochs, lr)
    
    loss_histories[city] = loss_history
    models[city] = lewis
    

    split = 'test'
    test_data = ArgoverseDataset(city = city, split = split)
    preds = pred(lewis, test_data)
    m, s = get_mean_std(city=city, split="train")
    preds = preds.detach().numpy()
    preds = reverse_std(m, s, preds)
    predictions[city] = preds
    
